# Distillation With Fine Tuning - Make It Cost-Effective

**Objective**

Zava's chatbot is now polite and helpful. But it uses GPT-4.1, which is is too , slow, and expensive for the simple task assigned to it. _Can we get the same behavior, with comparable quality, using a smaller model that would be faster and cheaper?

**Process**

In this notebook, we'll explore _distillation_, the process by which we use a larger model (GPT-4.1) to teach a smaller model (e.g., GPT-4.1-nano) to do a specific task, with comparable accuracy. It achieves this by trading off the breadth of knowledge of larger models for a _narrower intelligence_ that is more cost-effective. This technique is also referred to as _model compression_ since it effectively reduces the size of model used in the solution. The figure shows the basic workflow we will cover in this lab.

_This lab is adapted from the Distilling Sarcasm Lab created by Dave Voutila_.

![SFT](./../../../docs/slides/16.png)

---

## 1. Lab Objectives

Let's recap our journey so far, in customizing the tone and style of our selected base model (GPT-4.1). With each step, we are improving the quality of responses (tone, style) while optimizing for cost and latency.

| Journey | Outcome | 
|:---|:---|
| Prompt Engineering | No changes to GPT-4.1, but limited few-shot examples and longer prompt lengths | 
| Supervised Fine Tuning | Retrained GPT-4.1 with many Zava Q&A samples - shorter prompts, right tone but slow & expensive| 
| Distillation | Have GPT-4.1 train smaller, cheaper model - shorter prompts, right tone AND cost-effective | 

**By the end of this lab you should**
- Have a cost-effective model that delivers the required tone & style 
- Understand how _graders_ work to evaluate quality of model responses
- Understand how _distillation_ works to transfer knowledge from "teacher" to "student"


---

## 2. Lab Setup

**Pre-Requisites**

1. You have a deployed Azure AI Foundry project - with a GPT-4.1 model deployed
1. You have a .env file - with environment variables updated for the project
1. You authenticated with Azure from GitHub Codespaces - using `az login` in the terminal

**Process**


### 2.1 Deploy More Models

In this lab we'll also learn about graders and how to evaluate different models for the quality of their response, before selecting our teacher and student _candidates_ for distillation. Let's go for a mix of reasoning and non-reasoning models, and for both large and small versions where available.

1. Reasoning Models: o3, o3-mini, o4-mini
1. Non-Reasoning Models: gpt-4o, gpt-4o-mini, gpt-4.1, gpt-4.1-mini, gpt-4.1-nano

You will need sufficient quota (TPM) for these models - have a healthy subset of these if you can't deploy them all.

### 2.2 Set Environment Variables

Verify that you have set these environment variables, else correct that by updating `.env` and restarting kernel to refresh.

In [1]:
import os
required_env_vars = [
    "AZURE_OPENAI_ENDPOINT",
    "AZURE_OPENAI_API_KEY",
    "AZURE_SUBSCRIPTION_ID",
    "AZURE_RESOURCE_GROUP",
    "AZURE_AOAI_ACCOUNT"
]

missing_vars = [var for var in required_env_vars if not os.environ.get(var)]
if missing_vars:
    raise EnvironmentError(f"Missing required environment variables: {', '.join(missing_vars)}")
else:
    print("All required environment variables are set.")

All required environment variables are set.


### 2.3 Check Libraries

Make sure the required Python libraries were installed. This should be done automatically when using GitHub Codespaces from this repository

In [3]:
try:
    import pandas
    import openai
    import dotenv
    import azure.identity
    import azure.mgmt.cognitiveservices
    import IPython
    import uuid
    import eval_utils
    print("All required packages are available!")
except ImportError as e:
    print(f"Missing required package: {e}")

All required packages are available!


### 2.4 Setup Azure OpenAI Client

In [4]:
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv(override=True)

client = AzureOpenAI(
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version="2025-04-01-preview",
)

In [ ]:
## Create a unique identifier as a run suffix to avoid name collisions over multiple runs
import uuid
UNIQUE_ID = str(uuid.uuid4()).split("-")[0]
print(f"Run ID: {UNIQUE_ID}")

---

## 3. Evaluation-Driven Development

We need a *gold standard* sample, not to train our model on, but to measure
the efficacy of our grader.

We'll use some pre-canned politeness examples from the **Azure OpenAI** tutorial
on [fine-tuning a GPT model](https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/fine-tune?tabs=command-line).

The [baseline.jsonl](./baseline.jsonl) file contains the prompts from that
tutorial, but decomposed into a simple format of question/answer pairs.

In [ ]:
# Peek at our baseline *gold standard* dataset.
import json

rows = 0
with open("./baseline.jsonl", "r") as f:
    for line in f.readlines():
        print(json.loads(line))
        rows += 1
        if rows >= 3:
            break

## 1. Grading the Grader
The Grader is the lynchpin here, so it *must* be effective. 

Specifically, we'll use a **Score Model Grader** (aka `score_model` via the API).
It's job is to take a *prompt* and use it to derive a numeric *score*. We define
to decide if the score is "passing" or "failing", which we'll use as a simple
signal of quality.

> ℹ️ **Aside: Garbage In, Garbage Out**
> If you cannot explain to a reasoning model like `o3` how to identify high and low
> quality responses based on your objective, you have a *shaky foundation* from
> which to build upon.
>
> But keep in mind, `o3` is a *reasoning* model. You do *not* need to give it
> fine-grained instructions, but you should give it a *clear objective*.



### 📃 Preparing our first Evaluation
We'll start with the definition of the prompt for our grader (`GRADER_PROMPT`).

Things to note about this prompt:
- We let the grader know how we want it to behave, in this case as a politeness
  detector and fact checker.
- We inform it of what the input data looks like.
- We give it a rubrik for scoring each input.
- We give it a clear schema for its output to match what a Score Model Grader
  must output for the Evaluation system to interpret.

And that's it. In this case, we don't need examples, we rely on the reasoning
model to decide how to interpret politeness, etc.

In [ ]:
# We'll use the most capable reasoning model as our grader.
GRADER_MODEL = "o3"

# A simple score grader that quantifies politeness while accounting for accuracy
# of the model's response. We want the model to answer the question *correctly*
# while adjusting their tone. Incorrect answers are heavily penalized.
GRADER_PROMPT = """
You are an expert in politeness detection, while also being a good fact checker.

You'll be given a conversation in the form of a question and answer. 

## Scoring Criteria
Judge the answer using two metrics.

### Metric 1: How polite is the response?
On a scale from 1 to 10 inclusive with 1 being no politeness detected and 10 being
the most polite response ever, assign an integer value within that range based
on your assessment.

### Metric 2: How accurate is the response?
Assign a 1 if the response is factually correct. Assign a 0 for this metric if it's
incorrect or contains innacuracies.

### Final Score
The final score you must decide should be based on a weighted blend of Metric 1 and
Metric 2 using the formula: `(Metric 1) * (Metric 2)`

This means that if Metric 2 is zero, the final score must be zero.

## Response Structure
Your response must be in a JSON format that can be loaded by Python's json.loads()
function. It must resemble the following:

```
{
  "steps": [
    { 
      "description": <one sentence describing your reasoning for Metric 1>", 
      "result": <string representation of Metric 1's score> 
    },
    { 
      "description": <one sentence describing your reasoning for Metric 1>", 
      "result": <string representation of Metric 1's score> 
    }
  ],
  "result": <floating point value of the Final Score>
}

## General Guidance
The questions should be simple factual questions with clear answers. Deep research is
not required.
```
"""

Now we need to stage our baseline data in the Azure OpenAI service so the grader can
access and score each input.

Using the SDK, it's a simple file upload, but with a purpose of `evals`.

A file used for `evals` ideally is in JSONL format, but does **not** need to be in a
chat completions format. We can just use that `{ "question": "?", "answer": "..." }`
format from our baseline file without any further data engineering.


In [ ]:
# We're going to first evaluate our grader using a human-curated dataset.
# In this case, these are the examples from our fine-tuning tutorial. Let's pretend
# we know what the scores for these _should_ be.
grader_eval_file = None
with open("./baseline.jsonl", "rb") as f:
    grader_eval_file = client.files.create(purpose="evals", file=f)
    grader_eval_file = client.files.wait_for_processing(grader_eval_file.id)

print(f"Created eval file:\n{grader_eval_file.to_json(indent=2)}")


### 🏗️ Constructing the Grader
For evaluating the grader, we'll be giving it pre-canned prompts for it to score.

To do this, we have to provide some templating:

1. We give the system a template (`INPUT`) to plug in data from our baseline
   jsonl to construct a ficticious prompt from a model.
2. We provide a schema to describe the shape of our test data (the baseline file)
   in `SCHEMA`.
3. Lastly, we define the testing criteria (`TESTING_CRITERIA`) which takes our
   prompt template (`INPUT`), the name of our grader model (in Azure OpenAI, this
   is the _deployment name_ of the model to use), and the scoring details.

This is a lot of data, so it's important to take a moment and wrap your head around
this stuff. Remember, this is the _simple_ version of a Score Model Grader! Simple
being we're not generating prompts, we're just using a data file to populate a
template.

So, for example, if we have a row from `baseline.jsonl` that looks like:

```json
{ "question": "Who spilled coffee on their desk today?", "answer": "Dave" }
```

The _actual_ prompt that will be shown to the _grader_ will be:

```json
[
   { "role": "system", "content": "You are an expert in politeness detection,..." },
   { "role": "user", "content": "\nQ: Who spilled coffee on their desk today?\nA: Dave\n" }
]
```

> ℹ️ Once you know how to reason about this yourself, you can use something like the
> Azure OpenAI Chat Playground to manually test your grader! Just set the system
> prompt to your grader prompt and then provide the _user_ content like in the
> example above.

In [ ]:
# Now we define our Evaluation for validating our choice in grader prompt and model.

# The entire user prompt is data driven from the file. No generation is done using
# a model in this case, just simple string substitution using this pattern. This
# means we directly reference the two fields in our baseline.jsonl file.
USER_PROMPT = """
Q: {{item.question}}
A: {{item.answer}}
"""
INPUT = [
    {
        "type": "message",
        "role": "system",
        "content": { "type": "input_text", "text": GRADER_PROMPT }
    },
    {
        "type": "message",
        "role": "user",
        "content": { "type": "input_text", "text": USER_PROMPT }
    }
]

# We need to describe what our evaluation dataset looks like.
SCHEMA = {
    "type": "object",
    "properties": {
        "question": { "type": "string" },
        "answer": { "type": "string" },
    }
}
DATA_SOURCE = {
    "item_schema": SCHEMA,
    "include_sample_schema": False,
    "type": "custom",
}

# Lastly, we define test criteria that combines all the above.
TESTING_CRITERIA = {
    "name": "Auto politeness Grader",
    "type": "score_model",
    "model": GRADER_MODEL,
    "input": INPUT,
    "range": [1.0, 10.0],    # Our grader scores in a range from 1 to 10
    "pass_threshold": 4.0,   # Let's say a 4 is "passing" for now.
}

### 👨‍🔬 Putting it together into an Evaluation
We have our data source defined (`DATA_SOURCE`) and our testing criteria that defines
our grader (`TESTING_CRITERIA`). Now we have what we need to construct an Evaluation.

> An evaluation can contain multiple testing criteria (i.e. graders), but in our case
> we just use the one above. Just a heads up on why we're giving it a list here.


In [ ]:
# We've set up the parameters for our Eval, now we create it via the API.
grader_eval = client.evals.create(
    name=f"politeness-baseline-{UNIQUE_ENOUGH_KEY}",
    data_source_config=DATA_SOURCE,
    testing_criteria=[TESTING_CRITERIA],
)

print(f"⚖️ Submitted grader evaluation {grader_eval.id}.")


Oddly, we're **not done!** 

We defined the Evaluation, but each Evaluation needs a test Run. The Evaluation defines
the general terms, testing criteria, etc., but since we might want to vary some test
parameters, we actually need a **Run** to get anything done.

Astute readers may have noticed that _we never specified we want to use baseline.jsonl._

At the Run-level, we provide the specific test file. This lets you separate out the
schema definition of a test file from the data itself, so if you had multiple files
you wanted to test, you could create multiple runs.

We define `RUN_DATA_SOURCE` below to specify, by _file id_ which file we want this Run
to use for data driving our test.

In [ ]:
# Our evaluation needs a test run. This is where we let it know to use our
# "gold standard" file (baseline.jsonl) to test our grader.
RUN_DATA_SOURCE = {
    "type": "jsonl",
    "source": { "type": "file_id", "id": grader_eval_file.id }
}
grader_run = client.evals.runs.create(
    name=f"politeness-grader-{GRADER_MODEL}",
    eval_id=grader_eval.id,
    data_source=RUN_DATA_SOURCE,
)
print(f"🏃‍➡️ Submitted run {grader_run.id} for {grader_eval.id}.")

Nothing is instaneous in life, including Evals. Let's wait for our Run to complete.

We can do this by polling the status of the Run itself. (Note: we don't poll the Eval.)

In [ ]:
# An Eval Run takes time to complete. Let's actively wait for it to finish before continuing.
from IPython.display import clear_output
import time

start_time = time.time()

grader_run = client.evals.runs.retrieve(eval_id=grader_eval.id, run_id=grader_run.id)
while grader_run.status not in ["completed", "failed"]:
    time.sleep(5)
    clear_output(wait=True)

    grader_run = client.evals.runs.retrieve(eval_id=grader_eval.id, run_id=grader_run.id)
    now = time.time()
    mins, secs = int((now - start_time) // 60), int((now - start_time) % 60)
    print(f"⏱️ Elapsed time: {mins} minutes {secs} seconds")

print(f"🏁 Run {grader_run.id}: {grader_run.status}!")

### 📊 Viewing and Interpreting Results
Our Run completed...so now what?

You can view the results in Azure AI Foundry, or stay in the comfort of this
notebook 😉. Let's stay in the notebook.

There's a help script provided for rendering the results of all Runs for a given
list of Evaluations. It will render two things:

1. The pass percentage of each Run (i.e. % of prompts scoring above our provided
   pass threshold).
2. A histogram of individual scores from a Run letting us see how the score
   distribution looks to see if we're generating excellent (well-above passing)
   results or just barely passing. (_"C's get degrees,"_ as they say! 😜)

In [ ]:
# We've got a handy script for rendering the results from an Evaluations Runs. Let's
# eyeball this stuff. It uses the Evals API to retrieve the scores and plot them.
from eval_utils import display_evaluation_summary

display_evaluation_summary(client, [grader_eval.id], x_range=(0, 10))

# We should see that our grader generally thinks our "gold standard" is pretty
# polite. This is where we'd iterate on tuning the grader, making sure we
# clearly capture features for it to score, etc. We're keeping it simple for now.

### 🤔 So how's it looking?
Ok! Not bad if we consider our *gold standard* data as generally "good" quality
politeness. The Grader thinks these are decent examples with nothing below a `2.0` in
this case and with the *p90* score being `6.0` (this varies with each run of the
notebook, but it tends to land around here).

## 2. Baseline Testing of our Base Models
At this point we've done two things (hopefully!):

1. Developed a Grader we feel is fit for our task.
2. Learned how to construct and submit an Eval and Run.

> ℹ️ If you aren't confident in (2), this is a good time to go re-read the above.

### Generating our Dataset
The real beauty of this type of distillation is we don't have to kill ourselves
to create a dataset! We really only need to generate the end-user's side of the
prompt and not the model's side, making this _so much simpler._

Since our agent usecase here is Cora🛒, we just need some simple questions a
user of our application might ask. A model like `GPT-4.1` can do this for us with
a prompt like:

```
Generate 100 question and answer pairs that might be used in a quiz game. Output
the pairs in JSONL with the following schema:

{ "item": { "question": <the question>, "answer": <the answer> } }

Try not to create duplicates!
```

> Seriously, this is how I created `qa.jsonl`. You can use Github Copilot right
> inside Visual Studio Code to generate it and also detect/remove duplicates
> until you have 500 rows.

Let's take a peek at the data in [qa.jsonl](./qa.jsonl):


In [ ]:
# We've previously generated 500 Question/Answer pairs. Note that they are only
# focused on factual answers. We're not trying to provide any politeness here, just
# facts...and honestly we're not going to use the answers anyways...but you might
# use them if you had another grader that checked for gold standard answers.
import json

qa = []
with open("./qa.jsonl", "r") as f:
    for line in f.readlines():
        qa.append(json.loads(line))

print(f"Number of Q/A pairs: {len(qa)}")
for i in range(3):
    print(qa[i])

We now need to split this into two sets:

1. A set for **baseline** testing (`qa_baseline`)
2. A set for **validation** after fine-tuning (`qa_validation`)

A simple 50/50 split here is good because the point of the validation set here is to have
a totally _different_ test to use post-training that has _zero overlap_ with what we'll
be using to train our student.

In [ ]:
# Now let's split these into two sets: our baseline set and our validation set. We'll just split
# them in half for now. 

# First we'll randomize it to maybe prove a point that this isn't totally staged 😜
from random import shuffle
shuffle(qa)

# Now we split 50/50.
split_at = int(len(qa) / 2)
qa_baseline = qa[:split_at]
qa_validation = qa[split_at:]

# Check it.
print(f"{len(qa_baseline)} pairs for baseline testing, {len(qa_validation)} for validation.")

Let's upload our baseline dataset.

First, we'll write it to disk both to let you inspect it, but also because the OpenAI SDK
really prefers to upload data file files. 🙃

> Yeah, if you're a Python dev and know how to make Files-like objects from data in-memory,
> _sorry_, but the SDK will barf.

In [ ]:
# Now we'll upload our baseline dataset and prepare our Evaluation. We need to save the data
# to disk first for...reasons...because of the OpenAI SDK. That's fine.
filename = f"./politeness-baseline-{UNIQUE_ENOUGH_KEY}.jsonl"

with open(filename, "w") as f:
    for row in qa_baseline:
        json.dump(row, f)
        f.write("\n")

baseline_file = None
with open(filename, "rb") as f:
    baseline_file = client.files.create(purpose="evals", file=f)
    baseline_file = client.files.wait_for_processing(baseline_file.id)

print(f"Created baseline file:\n{baseline_file.to_json(indent=2)}")

### Defining our Baseline Evaluation
Most of this will look similar to above from section 1 where we evaluated our Grader.

There are some key differences:

1. The prompt template now uses `{{sample.output_text}}` because we'll be using the
   base model (the model under test) to generate the answer to the user's question.
2. Because we're mixing _both_ the provided dataset file with generated responses,
   we have to make some slight tweaks to our data source definition (`DATA_SOURCE`).

Other than that, this is pretty similar.

In [ ]:
# We'll now build out the Evaluation details. In this case, we'll *generate* responses
# using a base model, unlike before where we used the pre-canned results just to test
# the grader.

# We'll use a simple system prompt to show how distillation and fine-tuning let us
# get away without overly complex prompt engineering.
SYSTEM_PROMPT = "Cora is a Shopping Assistant who is polite and helpful. She always tries to suggest other things that might help the user."

# We'll use a flee of base models as our baseline, including `o3` (our grader).
BASE_MODELS = [
    "o3",
    "o4-mini",
    "gpt-4.1",
    "gpt-4.1-mini",
    "gpt-4.1-nano",
    "gpt-4o",
    "gpt-4o-mini"
]

# The prompt we'll grade will look like this pattern. Similar to before, but now we're
# going to use {{sample.output_text}} to substitute what the model under test generates.
USER_PROMPT = """
Q: {{item.question}}
A: {{sample.output_text}}
"""

# Input to our grader remains the same as before, but we reproduce it here for context.
INPUT = [
    {
        "type": "message",
        "role": "system",
        "content": { "type": "input_text", "text": GRADER_PROMPT }
    },
    {
        "type": "message",
        "role": "user",
        "content": { "type": "input_text", "text": USER_PROMPT }
    }
]

# The schema and data source are similar, but with one major difference noted below.
SCHEMA = {
    "type": "object",
    "properties": {
        "question": { "type": "string" },
        "answer": { "type": "string" },
    },
}
DATA_SOURCE = {
    "item_schema": SCHEMA, 
    "include_sample_schema": True, # Note this change! Needed for data gen.
    "type": "custom"
}

# Same testing criteria, reproduced again for context.
TESTING_CRITERIA = {
    "name": "Auto politeness Grader",
    "type": "score_model",
    "model": GRADER_MODEL,
    "input": INPUT,
    "range": [1.0, 10.0],
    "pass_threshold": 4.0,
}

# We create one Evaluation for *all* our base models. Each model is tested in a
# distinct Run that we'll define next.
baseline_eval = client.evals.create(
    name=f"sacarsm-baseline-{UNIQUE_ENOUGH_KEY}",
    data_source_config=DATA_SOURCE,
    testing_criteria=[TESTING_CRITERIA]
)
print(f"⚖️ Created baseline eval {baseline_eval.id}")



Now instead of a single Run, we submit _one Run per base model_.

This part is **new**, so let's look at it closely.

Each Run has its own data source defined. Like with the Grader evaluation where
we finally said _which_ file to use for a test dataset, we're not providing both
a prompt template _and_ a reference to the test dataset.

1. `source` -- looks like what we did previously, referencing our test data by
   file id.
2. `input_messages` -- provides our prompt template, looking similar to how we
   defined the Grader prompt previously. Note, however, we're now using our
   `SYSTEM_PROMPT` (the simple 1-liner Cora one) and wiring in the test data
   as the user's input.

> ℹ️ An astute reader will notice _we're not using the `answer` field_ from our
> test data. Yup! In this demo, we don't. You _could_ extend the grader to use
> that as "ground truth" for scoring the accuracy. Exercise left to you, my
> friend!

> ⚠️ Attention!
> There's one thing to call out and that's the `sampling_params`. This lets us
> provide tuning of the chat completion parameters to the model under test. In
> this case, we're tuning the max completion tokens.
> 
> However, there are two things to note:
> 1. We use a different value depending on if it's a reasoning model being tested
>    or if it's a GPT model. (I'm not sure if this matters?)
> 2. More importantly, Azure OpenAI has a 🐛bug where we need to specificy it as
>    `max_completions_tokens` and not `max_completion_tokens`. Note the lack of
>    the `s`.

In [ ]:
# Each run gets its own data source definition as it needs to specify a different
# model deployment to use for generation. The template is the prompt template
# sent to the model under test. It uses the simple Cora system prompt and for
# the user input, we use the "question" from the baseline Q&A data file.
baseline_runs = []
for model in BASE_MODELS:
    RUN_DATA_SOURCE = {
        "type": "completions",
        "model": model,
        "source": { "type": "file_id", "id": baseline_file.id },
        "input_messages": {
            "type": "template",
            "template": [
                { 
                    "type": "message", 
                    "role": "system", 
                    "content": { "type": "input_text", "text": SYSTEM_PROMPT },
                },
                { 
                    "type": "message", 
                    "role": "user", 
                    "content": { "type": "input_text", "text": "{{item.question}}" },
                },
            ],
        },
        "sampling_params": { "max_completions_tokens": 20_000 } if model.startswith("o") else { "max_completions_tokens": 100 }, # XXX
    }
    run = client.evals.runs.create(
        name=f"{model}-{UNIQUE_ENOUGH_KEY}", 
        eval_id=baseline_eval.id,
        data_source=RUN_DATA_SOURCE, 
    )
    print(f"🏃‍➡️ Created run {run.id} for eval {baseline_eval.id}")
    baseline_runs.append(run)

Waiting is always the hardest part. What can I say?

This can take 15-20 minutes depending on your TPM limits. Maybe more. Maybe less.

In [ ]:
# We now have to wait for our half-dozen or so Runs to finish. Twiddle your thumbs a bit!
from IPython.display import clear_output
import time

start_time = time.time()

while any([r.status not in ["completed", "failed"] for r in baseline_runs]):
    time.sleep(10)
    clear_output(wait=True)

    for i in range(len(baseline_runs)):
        baseline_runs[i] = client.evals.runs.retrieve(eval_id=baseline_eval.id, run_id=baseline_runs[i].id)
        print(f"🏃‍➡️ Run {baseline_runs[i].name}: {baseline_runs[i].status}")
    
    now = time.time()
    print("⏱️ Elapsed time: {} minutes {} seconds".format(int((now - start_time) // 60), int((now - start_time) % 60)))

print(f"🏁 All {len(baseline_runs)} runs completed!")

### Interpreting our Baseline Results
Now we get to see our winner and loser!

We'll use the same plotting function as before to look at _pass percentage_ and the
individual score distributions for each model.

What we want to do here is:
1. Identify the clear winner to designat our *teacher*.
2. Identify the clear loser to designate our *student*.

Recall our general hypothesis here is:

> A larger, more robust model will excel at our task out of the box at the cost of
> typically more $'s per token and often higher latency (time-to-first-token).
>
> A smaller, less robust model will perform poorly out of the box, but will provide
> a better $/token price-point and often much lower latency.
>
> The ideal model for our agent will have the lowest $/token and latency, while
> achieving acceptable scores.

Our goal, as a reminder, is to take that fast, cheap model and make it perform
**as well as the slower, more expensive model**.

> 😜 Better, Faster, Cheaper: Pick 3!

In [ ]:
# Let's visualize our evaluation and identify the best and worst performers.
display_evaluation_summary(client, [baseline_eval.id], x_range=(1, 10))


You'll notice we have a very clear winner and a very clear loser.

- `o3` -- consistently scores the highest both in terms of passing scores, but also
  generates generates a lot of `[6, 7, 8]` scores.
- `4.1-nano` -- consistently abysmal performance, with the most failing scores in
  the range of `[1, 2]`. Just an embarassment!

But let's hold that thought for a moment. Visually we can see this, but remember what
we want to do here is find not just the _best performing model_, but we need it's best
_example responses_.



## 4. 🧪 Distilling from the Teacher
Let's look at all our models again, this time via code, and find just the _excellent_
responses that scored `6.0` or higher.

> ℹ️ This part gets a bit technical! We'll be doing some data engineering on the fly
> as we analyze the excellent responses. This is maybe the more complex Python in this
> entire notebook as it works around some limitations with the Evaluations API, but
> buckle up and I promise it's worth it.

What we're going to do is query each Run, look at the Run's _individual results_,
collect just the "excellent" ones, and while we're doing so we'll be
_transforming them into chat completions_.

Then it's as simple as seeing which model had the most "excellent" scores and
declaring our winner.

In [ ]:
# We'll find the model that generated the most "excellent" (>= 6.0) examples of politeness.
CUTOFF = 6.0
HIGH_SCORES = {
    "o3": [],
    "o4-mini": [],
    "gpt-4.1": [],
    "gpt-4.1-mini": [],
    "gpt-4.1-nano": [],
    "gpt-4o": [],
    "gpt-4o-mini": [],
}

# Let's find our responses that were Excellent (at or above CUTOFF). We'll collect them
# and pre-format them into chat completions format to save time later.
#
# This part is honestly a bit tricky...we're extracting the prompts and responses for the
# model under test and *not* the prompts to the grader, so we have to do surgery. 🔪
for run in baseline_runs:
    pages = client.evals.runs.output_items.list(run.id, eval_id=baseline_eval.id).iter_pages()
    for page in pages:
        for item in page.data:
            # We only used 1 grader. If you use multiple, you should look for which ones you want.
            if not item.results:
                continue
            result = item.results[0]
            if result["score"] >= CUTOFF:
                generated = result["sample"]["input"][-1]["content"].strip().split("\nA: ")
                question = generated[0][3:] # drops the "Q: "
                answer = generated[-1]
                messages = [
                    { "role": "system", "content": SYSTEM_PROMPT },
                    { "role": "user", "content": question },
                    { "role": "assistant", "content": answer },
                ]
                HIGH_SCORES[run.model].append({ "messages": messages })

# Time to find the winner! Obviously, this is probably o3...
winning_model = ""
winning_cnt = 0
for key in HIGH_SCORES.keys():
    if len(HIGH_SCORES[key]) > winning_cnt:
        winning_model = key
        winning_cnt = len(HIGH_SCORES[key])
        
print(f"😜 Ok! Let's use {winning_model}. It had {winning_cnt} excellent responses.")

We now take _just the Teacher's excellent responses_ and construct our training data.

We should have enough for Supervised Fine Tuning if all goes well (based on previous testing of this notebook),
so let's give it a train/test split:

1. Split the data in-memory.
2. Write the training and validation data out as JSONL to disk.
3. Upload them via the Files API.

In [ ]:
# Before we go any further, let's turn our collected excellent responses into our
# training and validation fine-tuning datasets. Like before, we have to write these
# to disk and then upload them via the Files API.
training_filename = f"politeness-training-{UNIQUE_ENOUGH_KEY}.jsonl"
validation_filename = f"politeness-validation-{UNIQUE_ENOUGH_KEY}.jsonl"

# Make an 80/20 split to form our training/validation data.
split_at = int(len(HIGH_SCORES[winning_model]) * 0.80)
training_data = HIGH_SCORES[winning_model][:split_at]
validation_data = HIGH_SCORES[winning_model][split_at:]
print(f"Split into {len(training_data)} training / {len(validation_data)} validation rows.")

# Create and upload the training data.
with open(training_filename, "w") as f:
    for message in training_data:
        json.dump(message, f)
        f.write("\n")
with open(training_filename, "rb") as f:
    training_file = client.files.create(file=f, purpose="fine-tune")
    training_file = client.files.wait_for_processing(training_file.id)
print(f"🏋️‍♂️ Created training file:\n{training_file.to_json(indent=2)}")

# Create and upload the validation data.
with open(validation_filename, "w") as f:
    for message in validation_data:
        json.dump(message, f)
        f.write("\n")
with open(validation_filename, "rb") as f:
    validation_file = client.files.create(file=f, purpose="fine-tune")
    validation_file = client.files.wait_for_processing(validation_file.id)
print(f"📋 Created validation file:\n{validation_file.to_json(indent=2)}")

## 5. 🏋️‍♂️ Training the Student
We've got our training dataset, so let's get to fine-tuning!

We've chosen `4.1-nano` as the student because it's desperately in the need of some help as
shown by its benchmarking.

We'll create a job _suffx_ that identifies our teacher model for posterity and include that
unique-enough key from before so if you want to run this notebook again you can have two
different fine-tuning jobs to compare.

> ℹ️ Look at the use of `extra_body` as a parameter! We're telling Azure OpenAI to try
> using Global Training for our job. If you want to learn more about Global Training, check
> out the orginal [announcement](https://aka.ms/Build25/FTGlobalAndDev) from **Build 2025**.

Note that we're not tuning hyper-parameters here for our job and just using defaults. That
is an exercise left for you, dear reader!

In [ ]:
# Now we start training! Submit our fine-tuning job to teach 4.1-nano new tricks.
TEACHER_MODEL = winning_model
STUDENT_MODEL = "gpt-4.1-nano-2025-04-14"
SUFFIX = f"{TEACHER_MODEL}-politeness-{UNIQUE_ENOUGH_KEY}".replace(".", "") # '.' is a reserved character 😜

job = client.fine_tuning.jobs.create(
    model=STUDENT_MODEL,
    suffix=SUFFIX,
    training_file=training_file.id,
    validation_file=validation_file.id,
    extra_body={ "trainingType": "globalstandard" }
)
print(f"👨‍🔬 Created fine-tuning job:\n{job.to_json(indent=2)}")

Fine-tuning, like Evaluations, take time. Now's a good time to go catch up on emails ✉️,
walk your dog 🐕, or take a nap 😴. We can poll the training status to know when we can
proceed further in the notebook.

In [ ]:
# Wait for our FT job to complete. You may want to go work on some other tasks for now 😜
from IPython.display import clear_output
import time

start_time = time.time()

status = job.status
while status not in ["succeeded", "failed", "cancelled"]:
    time.sleep(10)
    job = client.fine_tuning.jobs.retrieve(job.id)
    status = job.status
    clear_output(wait=True)
    print(f"👨‍🔬 Job {job.id}: {status}")
    print("⏱️ Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))

if status == "succeeded":
    print(f"🏁 Fine-tuning finished!")
else:
    raise RuntimeError(f"Fine-tuning job did not complete successfully (status={status})")

Once the job completes, we should eyeball the metrics. Here's what an example
should look like:

`TODO: INSERT IMAGE`

If our `4.1-nano` model is learning, you should see the _training loss_ decreasing.
This gives us a sense of if the model learned something, but it **doesn't** tell us
if the model is fit for production!

## 6. 🧑‍⚖️ Judging our Student against its Peers
Shipping this new model to production right now would be truly a YOLO moment and I
cannot recommend it.

What we really need to do now is go back and benchmark it against the original
`4.1-nano` base model.

Remember how we split that initial Q&A dataset into two parts? The `qa_validation`
file contains unseen questions for our models that could not possible appear in the
training data, so it's the perfect check to answer the question if we actually
moved the needle.

### 🚢 Deploying our New Model
Azure OpenAI requires you to "deploy" a model in order to have an endpoint to call
it, so let's do that.

We need to reach for a different SDK, though, but no worries! The Azure Cognitive
Services SDK let's us talk to the Azure OpenAI control plane to deploy things
without having to leave this comfy notebook.

> ⚠️ The code below uses a `DefaultAzureCredential`. The easiest way to make sure
> one exists is to have installed and authenticated the Azure CLI tooling.
>
> See: https://learn.microsoft.com/en-us/cli/azure/install-azure-cli?view=azure-cli-latest

We'll use the new Azure OpenAI `Developer Tier` available for Fine-Tuned models as
it's purpose built for model candidate evaluation: you only pay per token at the
same base model rates! (Learn more [here](https://aka.ms/Build25/FTGlobalAndDev).)



In [ ]:
# DELETE THIS LATER - THIS IS A HACK TO GET ENV VAR INTO RUNNING NOTEBOOK
os.environ["AZURE_SUBSCRIPTION_ID"] = ""
os.environ["AZURE_RESOURCE_GROUP"] = "nitya-aitour26-ftbrk-rg"
os.environ["AZURE_AOAI_ACCOUNT"] = "nitya-aitour26-ftbrk-resource"

In [ ]:
# Now we need to deploy our fine-tuned model. We'll use Developer Tier to keep
# costs under control for evaluation.

# We can't do this with the OpenAI SDK, so we need to reach for the Azure SDK.
from azure.identity import DefaultAzureCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient

cogsvc_client = CognitiveServicesManagementClient(
    credential=DefaultAzureCredential(),
    subscription_id=os.environ.get("AZURE_SUBSCRIPTION_ID")
)

# Define our Deployment. Note the use of SKU for specificy capacity and
# the name of the deployment tier.
DEPLOYMENT_NAME = f"politeness-distilled-{SUFFIX}"
DEPLOYMENT = {
    "properties": {
        "model": { 
            "format": "OpenAI", 
            "name": job.fine_tuned_model, 
            "version": "1" 
        },
    },
    "sku": { 
        "capacity": 250, 
        "name": "DeveloperTier" 
    },
}

# Submit the request for provisioning. This may take a few minutes, so we
# poll for updates. If it already exists, this should return quickly. Since
# we're deploying a 4.1-nano model, this should only take 3-5 minutes tops.
deployment = cogsvc_client.deployments.begin_create_or_update(
    resource_group_name=os.environ.get("AZURE_RESOURCE_GROUP"),
    account_name=os.environ.get("AZURE_AOAI_ACCOUNT"),
    deployment_name=DEPLOYMENT_NAME,
    deployment=DEPLOYMENT,
)
print(f"🛳️ Submitted deployment {deployment}")

Deployments, like Evaluations and Training, are not instaneous, but they are typically
a lot faster! `4.1-nano` typically will deploy in 3-4 minutes, so maybe don't walk
away from your desk just yet 😜.

In [ ]:
# Wait for our deployment to finish provisioning.
start_time = time.time()
status = deployment.status()

while status not in ["Succeeded", "Failed"]:
    deployment.wait(5)
    status = deployment.status()
    clear_output(wait=True)
    print(f"🛳️ Provisioning {DEPLOYMENT_NAME}: {status}")
    print("⏱️Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))

print(f"🏁 Provisioning finished!")


### ⬆️ Uploading the Validation Data
This should be old-hat by now! We'll take the in-memory validatation data (`qa_validation`)
and create a dataset via the Files API.

In [ ]:
# Now we'll upload our post-training validation dataset and prepare our final Evaluation.
# We need to save the data to disk first, again for...reasons.
filename = f"./politeness-posttraining-{UNIQUE_ENOUGH_KEY}.jsonl"

with open(filename, "w") as f:
    for row in qa_validation:
        json.dump(row, f)
        f.write("\n")

posttraining_file = None
with open(filename, "rb") as f:
    posttraining_file = client.files.create(purpose="evals", file=f)
    posttraining_file = client.files.wait_for_processing(posttraining_file.id)


### 🏃 Creating Runs to make our Student Compete with the Base Model
Here's where Evaluations really shine ✨!

We _could_ reuse our baseline evaluation from before and just add new Runs so all the
results are in one place and easily viewable.

In this case, we'll keep it simple and create a new Evaluation primarily because the
visualization code already has color-coding based on Eval, not Run and I'm too lazy
to rewrite it further 😜.

Most of this is repeat, in fact we could have reused thigns like `DATA_SOURCE`,
`TESTING_CRITERIA`, etc. but restate them below for completeness. Same can be said
for the Runs.

*However*, we'll focus on just comparing our new model to `4.1-nano` and also add in
`4.1` as sort of a control. We're testing with totally new data here, unseen by any
of our models, so what we expect to see is:

1. The base `4.1-nano` performs poorly again.
2. The base `4.1` performs decently and on-par with the previous baseline.
3. The new student model outperforms ideally both, but at least `4.1-nano`.


In [ ]:
# Now we create a final Eval using our post-training dataset that doesn't overlap with the
# original distillation and training dataset. This lets us judge our new model based on
# data it hasn't seen before. We'll also through in one of our better performing base
# models as a control.
POST_EVAL_MODELS = [
    DEPLOYMENT_NAME,
    "gpt-4.1-nano",
    "gpt-4.1",
]

# SCHEMA, GRADER_MODEL, and INPUT are re-used from our previous Evaluation definition,
# but let's restate the source and testing criteria again.
DATA_SOURCE = {
    "item_schema": SCHEMA, 
    "include_sample_schema": True, # Note this change! Needed for data gen.
    "type": "custom"
}
TESTING_CRITERIA = {
    "name": "Auto politeness Grader",
    "type": "score_model",
    "model": GRADER_MODEL,
    "input": INPUT,
    "range": [1.0, 10.0],
    "pass_threshold": 4.0,
}
posttraining_eval = client.evals.create(
    name=f"politeness-posttrain-evaluation-{UNIQUE_ENOUGH_KEY}",
    data_source_config=DATA_SOURCE,
    testing_criteria=[TESTING_CRITERIA]
)
print(f"Created eval {posttraining_eval.id}")

# Now add our runs.
postraining_runs = []
for model in POST_EVAL_MODELS:
    RUN_DATA_SOURCE = {
        "type": "completions",
        "model": model,
        "source": { "type": "file_id", "id": posttraining_file.id },
        "input_messages": {
            "type": "template",
            "template": [
                { 
                    "type": "message", 
                    "role": "system", 
                    "content": { "type": "input_text", "text": SYSTEM_PROMPT },
                },
                { 
                    "type": "message", 
                    "role": "user", 
                    "content": { "type": "input_text", "text": "{{item.question}}" },
                },
            ],
        },
        "sampling_params": { "max_completions_tokens": 100 }, # XXX again, note the purposeful typo
    }
    run = client.evals.runs.create(
        name=f"{model}-{UNIQUE_ENOUGH_KEY}", 
        eval_id=posttraining_eval.id,
        data_source=RUN_DATA_SOURCE, 
    )
    print(f"🏃‍➡️ Created run {run.id} for {posttraining_eval.id}")
    postraining_runs.append(run)


Again, we wait! ⏱️

In [ ]:
# Again, we wait for our runs to finish.
start_time = time.time()

while any([r.status not in ["completed", "failed"] for r in postraining_runs]):
    time.sleep(10)
    clear_output(wait=True)

    for i in range(len(postraining_runs)):
        postraining_runs[i] = client.evals.runs.retrieve(eval_id=posttraining_eval.id, run_id=postraining_runs[i].id)
        print(f"🏃‍➡️ Run {postraining_runs[i].name}: {postraining_runs[i].status}")
    
    now = time.time()
    print("⏱️ Elapsed time: {} minutes {} seconds".format(int((now - start_time) // 60), int((now - start_time) % 60)))

print(f"🏁 All {len(postraining_runs)} runs completed!")

### 📊 Interpreting the Post-Training Results
Let's first look at _just the new Evaluation_ to see if our new model outperforms its
competition (`4.1-nano` and `4.1`).

In [ ]:
# Let's visualize our post-training evaluation. Fingers crossed!
display_evaluation_summary(client, [posttraining_eval.id], x_range=(1, 10))

In [ ]:
# And now let's put it all together.
# Let's visualize our post-training evaluation. Fingers crossed!
display_evaluation_summary(client, [baseline_eval.id, posttraining_eval.id], x_range=(1, 10))

## 🥳 7. Celebrate or Cry
It _should_ be time to celebrate if all went well!

We can clearly see our new model `politeness-distill` is outperforming all but `o3` in terms
of pass-percentage, but maybe most importantly has adopted the high-scoring ability of
`o3`, generating scores at `6.0` or above!


## 🔚 8. Conclusion
In this notebook we demonstrated **distillation** using Azure OpenAI *Evaluations*
and *Fine Tuning* features.

We used an objective of *adjusting the tone* of a model to meet our needs, in this
case making its responses polite, while preserving accuracy in results, and
*distilled* the native capabilities of a state-of-the-art reasoning model (`o3`)
into a much smaller, non-reasoning model (`4.1-nano`) to let our agent or app
use the smallest model possible while:

- 🤑 minimizing per-token costs
- 🏎️ improve performance (latency)

We did all this:

- without creating training data directly
- without knowing the ideal student model
- only by knowing how to define our Grader

So to wrap it all up:

1. We described the ideal state to our complex reasoning model in the form of
   a few samples we feel are ideal.
2. We described to the reasoning model (grader) how to judge those examples
   to measure their quality.
3. We let Evaluations and Fine Tuning do the rest!




## 🧹 9. Cleanup
If you want to clean up everthing from this notebook, set `cleanup = True` below.

In [ ]:
# Want to clean up your mess? 
cleanup = False

if cleanup:
    pass # TODO: add in calls to nuke everything